In [1]:
#!/usr/bin/env python

import os
import numpy as np
import pandas as pd
from datetime import datetime

# ============================================================
# CONFIG: FILE PATHS (ADJUST TO YOUR MACHINE)
# ============================================================

UNIVERSE_TRADABLE = r"C:\TWS API\source\pythonclient\TradingIdeas\Momentum System\12-tradable_sp500_universe\12-tradable_sp500_universe.parquet"
UNIVERSE_FULL     = r"C:\TWS API\source\pythonclient\TradingIdeas\Momentum System\11-universe_with_jump90\11-universe_with_jump90.parquet"
SPY_REGIME_FILE   = r"C:\TWS API\source\pythonclient\TradingIdeas\Momentum System\8-SPY_200DMA_market_regime\8-SPY_200DMA_regime.parquet"

OUTPUT_DIR = "./deep_robustness_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

START_TRADING = pd.Timestamp("1999-01-01")   # same as your main system
INITIAL_CAPITAL = 500_000
REBALANCE_DAY = "Wednesday"

TRADING_DAYS_PER_YEAR = 252


# ============================================================
# METRIC HELPERS
# ============================================================

def cagr_from_curve(eq: pd.Series):
    eq = eq.dropna()
    if len(eq) < 2:
        return np.nan
    years = len(eq) / TRADING_DAYS_PER_YEAR
    if years <= 0 or eq.iloc[0] <= 0:
        return np.nan
    return (eq.iloc[-1] / eq.iloc[0]) ** (1 / years) - 1


def max_drawdown(eq: pd.Series):
    eq = eq.dropna()
    if len(eq) == 0:
        return np.nan
    peak = eq.cummax()
    dd = eq / peak - 1
    return dd.min()


def sharpe_ratio(returns: pd.Series):
    r = returns.dropna()
    if len(r) < 2 or r.std() == 0:
        return np.nan
    return r.mean() / r.std() * np.sqrt(TRADING_DAYS_PER_YEAR)


def sortino_ratio(returns: pd.Series):
    r = returns.dropna()
    downside = r[r < 0]
    if len(downside) == 0 or downside.std() == 0:
        return np.nan
    return r.mean() / downside.std() * np.sqrt(TRADING_DAYS_PER_YEAR)


def summarize_equity(name, eq: pd.Series):
    eq = eq.dropna()
    ret = eq.pct_change()
    return {
        "name": name,
        "CAGR_%": cagr_from_curve(eq) * 100,
        "MaxDD_%": max_drawdown(eq) * 100,
        "Sharpe": sharpe_ratio(ret),
        "Sortino": sortino_ratio(ret),
        "FinalValue": eq.iloc[-1],
        "StartDate": eq.index[0].date() if len(eq) else None,
        "EndDate": eq.index[-1].date() if len(eq) else None,
    }


# ============================================================
# LOAD DATA
# ============================================================

print("Loading tradable universe...")
trad = pd.read_parquet(UNIVERSE_TRADABLE)
trad["date"] = pd.to_datetime(trad["date"])

print("Loading full universe...")
full = pd.read_parquet(UNIVERSE_FULL)
full["date"] = pd.to_datetime(full["date"])

print("Loading SPY regime...")
spy_regime = (
    pd.read_parquet(SPY_REGIME_FILE)
    .reset_index()
    .rename(columns={"Date": "date"})
)
spy_regime["date"] = pd.to_datetime(spy_regime["date"])

# Merge SPY regime into both universes
merge_cols = ["date", "spy_close", "spy_ma200", "market_regime"]

trad = trad.merge(spy_regime[merge_cols], on="date", how="left")
full = full.merge(spy_regime[merge_cols], on="date", how="left")

print("Rows tradable:", len(trad))
print("Rows full    :", len(full))

# Make sure numeric columns are numeric
for df in (trad, full):
    for col in ["close_adj", "slope_adj"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")


# ============================================================
# HELPER: BUILD DAILY PRICE PANEL (date x ticker)
# ============================================================

def build_price_panel(df: pd.DataFrame):
    """
    Pivot to date x ticker price panel for close_adj.
    """
    pivot = (
        df
        .pivot_table(index="date", columns="ticker", values="close_adj")
        .sort_index()
    )
    return pivot


price_panel_trad = build_price_panel(trad)
price_panel_full = build_price_panel(full)

# daily returns (no forward fill to avoid look-ahead)
daily_returns_trad = price_panel_trad.pct_change(fill_method=None)
daily_returns_full = price_panel_full.pct_change(fill_method=None)


# ============================================================
# CORE SIMPLE MOMENTUM ENGINE (ROBUSTNESS HARNESS)
# ============================================================

def run_simple_momo(
    name: str,
    universe: str = "tradable",     # "tradable" or "full"
    top_percentile: float = 0.9,    # e.g. 0.8, 0.9, 0.95
    top_k: int | None = None,       # e.g. 20, or None to ignore
    slope_lag_days: int = 0,        # 0 = use same-day slope_adj; 1 = lag by 1 day; etc.
    lag_filters_by_days: int = 0,   # lag in_sp500, no_big_jump_90, above_ma100, market_regime
    exec_delay_days: int = 0,       # 0 = trade at close today, 1 = trade at next day's close
    min_history_days: int = 0,      # require at least this many trading days of price history
    require_spy_bull: bool = True,  # True = only hold when SPY regime is bull
    rebalance_weekday: str = "Wednesday",  # rebalance weekday name
):
    """
    Simplified weekly momentum system used *only* for robustness tests.

    Logic (per rebalance day):
      1. Pick universe: 'tradable' = 12-tradable_sp500_universe, 'full' = 11-universe_with_jump90.
      2. Build cross-section for that date (possibly lagging slopes & filters).
      3. Apply filters:
           - in_sp500 (if present and True)
           - no_big_jump_90 (if present and True)
           - above_ma100 (if present and True)
           - min_history_days of price data (if > 0)
           - SPY regime (if require_spy_bull == True)
      4. Rank remaining stocks by slope_adj (momentum).
      5. Selection:
           - If top_k is provided: choose min(top_k, number of candidates).
           - Else: choose all with slope_adj >= given top_percentile.
      6. Equal-weight each selected name, fully invested (no ATR sizing here).
      7. Execute trades either at same-day close (exec_delay_days=0) or next-day close (T+1).
      8. Track daily equity curve and stats from START_TRADING onward.
    """
    if universe == "tradable":
        df = trad.copy()
        price_panel = price_panel_trad
        daily_returns = daily_returns_trad
    elif universe == "full":
        df = full.copy()
        price_panel = price_panel_full
        daily_returns = daily_returns_full
    else:
        raise ValueError("universe must be 'tradable' or 'full'")

    # Restrict to dates with price info
    all_dates = price_panel.index
    all_dates = all_dates[all_dates >= START_TRADING]

    # For fast access: group by date
    df_by_date = {d: sub for d, sub in df.groupby("date")}

    # Precompute SPY regime series (bull/bear) and lag if needed
    spy_reg = (
        df[["date", "market_regime"]]
        .drop_duplicates("date")
        .set_index("date")
        ["market_regime"]
        .sort_index()
    )
    if lag_filters_by_days > 0:
        spy_reg = spy_reg.shift(lag_filters_by_days)

    # Precompute price history length per ticker/date (trading days)
    # We'll use price_panel to compute "days since first non-null close"
    history_len = (~price_panel.isna()).cumsum()

    # Rebalance dates = all dates with given weekday
    rebalance_dates = [d for d in all_dates if d.day_name() == rebalance_weekday]

    equity = INITIAL_CAPITAL
    positions = {}  # ticker -> weight
    equity_curve = []

    prev_date = None

    for current_date in all_dates:

        # First apply yesterday's positions to today's returns to evolve equity
        if prev_date is not None:
            if positions:
                day_ret = 0.0
                for t, w in positions.items():
                    if t not in daily_returns.columns:
                        continue
                    r = daily_returns.loc[current_date, t]
                    if pd.isna(r):
                        continue
                    day_ret += w * r
                equity *= (1.0 + day_ret)
            # else all cash → equity unchanged

        # Record equity
        equity_curve.append({"date": current_date, "equity": equity})

        # Rebalance on chosen weekday
        if current_date in rebalance_dates:

            # Determine decision date for signals (lagging slope & filters)
            decision_date = current_date
            if slope_lag_days > 0 or lag_filters_by_days > 0:
                # shift back by n trading days, not calendar days
                idx = np.searchsorted(all_dates, current_date) - max(slope_lag_days, lag_filters_by_days)
                if idx < 0:
                    # not enough history yet
                    pass
                else:
                    decision_date = all_dates[idx]

            day_df = df_by_date.get(decision_date)
            if day_df is None:
                # no universe data for that day
                positions = {}
            else:
                # Filter universe
                sub = day_df.copy()

                # Only allow stocks actually in the price panel on *execution* date
                # Execution date may be T or T+1
                exec_idx = np.searchsorted(all_dates, current_date) + exec_delay_days
                if exec_idx >= len(all_dates):
                    # past data end, no more trades
                    pass
                else:
                    exec_date = all_dates[exec_idx]

                    if min_history_days > 0:
                        # require at least 'min_history_days' trading days of price history by exec_date
                        for_hist = history_len.loc[:exec_date]
                        # map: ticker -> history length at exec_date
                        valid_hist = for_hist.iloc[-1]
                        sub = sub[sub["ticker"].map(lambda t: valid_hist.get(t, 0) >= min_history_days)]

                    # Basic filters (if present)
                    if "in_sp500" in sub.columns:
                        sub = sub[sub["in_sp500"] == True]

                    if "no_big_jump_90" in sub.columns:
                        sub = sub[sub["no_big_jump_90"] == True]

                    if "above_ma100" in sub.columns:
                        sub = sub[sub["above_ma100"] == True]

                    # SPY regime filter
                    if require_spy_bull:
                        spy_flag = spy_reg.loc[decision_date] if decision_date in spy_reg.index else np.nan
                        if not (pd.notna(spy_flag) and spy_flag > 0):
                            # regime not bull -> no positions
                            positions = {}
                            prev_date = current_date
                            continue

                    # Rank by slope_adj
                    sub = sub[sub["slope_adj"].notna()]
                    if len(sub) < 5:
                        positions = {}
                    else:
                        sub = sub.sort_values("slope_adj", ascending=False)

                        # Selection: either hard top_k or percentile
                        if top_k is not None:
                            sub_sel = sub.head(top_k)
                        else:
                            cutoff = sub["slope_adj"].quantile(top_percentile)
                            sub_sel = sub[sub["slope_adj"] >= cutoff]

                        if len(sub_sel) == 0:
                            positions = {}
                        else:
                            # Equal-weight among selected
                            tickers = list(sub_sel["ticker"].unique())
                            w = 1.0 / len(tickers)
                            positions = {t: w for t in tickers}

        prev_date = current_date

    eq = pd.DataFrame(equity_curve).set_index("date")["equity"]
    eq = eq[eq.index >= START_TRADING]
    stats = summarize_equity(name, eq)
    return eq, stats


# ============================================================
# DEFINE ROBUSTNESS SCENARIOS
# ============================================================

scenarios = []

# 1) Baseline-like tradable universe
scenarios.append(dict(
    name="baseline_tradable",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))

# 2) Full universe, no in_sp500 filter (stress survivors / delistings)
scenarios.append(dict(
    name="full_universe_no_sp500_filter",
    universe="full",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))

# 3) Tradable, slope lagged by 1 and 5 days
scenarios.append(dict(
    name="tradable_slope_lag1",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=1,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))
scenarios.append(dict(
    name="tradable_slope_lag5",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=5,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))

# 4) Lag filters (in_sp500, no_big_jump_90, above_ma100, market_regime) by 1 day
scenarios.append(dict(
    name="tradable_filters_lag1",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=1,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))

# 5) T+1 execution
scenarios.append(dict(
    name="tradable_exec_Tplus1",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=1,
    min_history_days=0,
    require_spy_bull=True,
))

# 6) Require at least 200 days of history before trading a stock
scenarios.append(dict(
    name="tradable_minHistory200",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=200,
    require_spy_bull=True,
))

# 7) Top 20 hard cap (equal-weight), tradable universe
scenarios.append(dict(
    name="tradable_top20",
    universe="tradable",
    top_percentile=0.0,   # ignored because top_k is set
    top_k=20,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))

# 8) Percentile variations: 0.8 and 0.95
scenarios.append(dict(
    name="tradable_p80",
    universe="tradable",
    top_percentile=0.8,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))
scenarios.append(dict(
    name="tradable_p95",
    universe="tradable",
    top_percentile=0.95,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=True,
))

# 9) No SPY regime filter (always on)
scenarios.append(dict(
    name="tradable_no_spy_filter",
    universe="tradable",
    top_percentile=0.9,
    top_k=None,
    slope_lag_days=0,
    lag_filters_by_days=0,
    exec_delay_days=0,
    min_history_days=0,
    require_spy_bull=False,
))


# ============================================================
# RUN ALL SCENARIOS
# ============================================================

all_stats = []
all_eq_curves = {}

for cfg in scenarios:
    name = cfg["name"]
    print(f"\n=== Running scenario: {name} ===")
    eq, stats = run_simple_momo(**cfg)
    all_eq_curves[name] = eq
    all_stats.append(stats)

    # Save equity curve for later inspection
    out_path = os.path.join(OUTPUT_DIR, f"equity_{name}.parquet")
    eq.reset_index().rename(columns={"equity": "portfolio_value"}).to_parquet(out_path, index=False)
    print(f"Saved equity curve → {out_path}")

# Build summary DataFrame
stats_df = pd.DataFrame(all_stats)
stats_df = stats_df[["name", "CAGR_%", "MaxDD_%", "Sharpe", "Sortino", "FinalValue", "StartDate", "EndDate"]]
stats_df = stats_df.sort_values("CAGR_%", ascending=False)

print("\n=== DEEP ROBUSTNESS SUMMARY ===")
print(stats_df.to_string(index=False, float_format=lambda x: f"{x:8.3f}" if isinstance(x, float) else str(x)))

summary_path = os.path.join(OUTPUT_DIR, "deep_robustness_summary.csv")
stats_df.to_csv(summary_path, index=False)
print(f"\nSummary saved → {summary_path}")


Loading tradable universe...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\TWS API\\source\\pythonclient\\TradingIdeas\\Momentum System\\12-tradable_sp500_universe\\12-tradable_sp500_universe.parquet'